In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr

In [5]:
# trim 2 dataframes to match
def trim(dataframe_1, dataframe_2):
    def process(df):
        df.sort_index(inplace=True)
        temp = df.iloc[:,0].str.strip()
        df.drop(columns=df.columns[0], inplace = True)
        df.sort_index(axis=1, inplace=True)
        df.insert(0,temp.name,temp)
        return df
    
    df1 = process(dataframe_1.copy())
    df2 = process(dataframe_2.copy())
    
    # match indexes
    mask = df1.iloc[:, 0].isin(df2.iloc[:, 0])
    df1 = df1[mask]
    mask = df2.iloc[:, 0].isin(df1.iloc[:, 0])
    df2 = df2[mask]
    
    mask = df1.columns.str.strip().isin(df2.columns.str.strip())
    df1 = df1.loc[:, mask]
    mask = df2.columns.str.strip().isin(df1.columns.str.strip())
    df2 = df2.loc[:, mask]
    
    return df1, df2

In [76]:
def mut_trim(mutation, dataframe):
    # initialize dataframe
    df = pd.DataFrame(columns = np.append(dataframe.columns, 'throwaway'))
    df.iloc[:,0] = np.append(dataframe.iloc[:,0], 'throwaway')
    df.iloc[:,1:] = 0
    
    # add mutations
    r = {key: i for i, key in enumerate(df.iloc[:, 0])}
    c = {key: i for i, key in enumerate(df.columns)}
    for i in range(len(mutation)): 
        df.iloc[r.get(mutation.iloc[i, 0], -1), c.get(mutation.iloc[i, -1], -1)] = 1
    
    return df.iloc[:-1, :-1]

In [7]:
shRNA = pd.read_csv(r"C:\Users\justi\Coding\Project\Han Xu\DepMap_data\DepMap_data\shRNA_Broad_Trimmed_Wei.csv")
shRNA

,Unnamed: 0,22RV1_PROSTATE,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,786O_KIDNEY,A1207_CENTRAL_NERVOUS_SYSTEM,A172_CENTRAL_NERVOUS_SYSTEM,A204_SOFT_TISSUE,A2058_SKIN,A549_LUNG,A673_BONE,...,UACC812_BREAST,WM115_SKIN,WM1799_SKIN,WM2664_SKIN,WM793_SKIN,WM88_SKIN,WM983B_SKIN,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,ZR751_BREAST
0,A1BG,0.137563,0.407828,-0.378410,-0.122246,0.059608,0.594396,-0.595518,0.200742,0.538545,...,-0.464406,0.047725,0.529830,-0.684247,0.248592,0.155388,0.589643,1.137463,-0.171007,-0.001838
1,A1CF,-1.307328,1.173509,-0.170096,2.225973,-0.091072,-0.175562,-2.284927,-0.030372,-0.743267,...,-0.874380,-1.222956,-0.422814,-2.264726,-0.239732,-2.279778,0.216230,-2.425942,0.511213,-0.160431
2,A2M,-0.193665,0.077275,0.202445,-0.220164,0.424465,0.459323,-0.287543,0.193018,0.589247,...,-0.151241,0.161408,0.168103,0.419712,0.609845,1.260653,-0.476685,0.442686,0.180620,0.500914
3,A2ML1,0.457342,0.463284,-0.687812,-0.348345,0.309870,0.563500,0.324843,0.711249,-0.098360,...,0.986545,-0.350801,0.169529,0.789282,0.423673,0.353363,-0.441035,-0.233830,0.056519,0.940200
4,A4GALT,-0.061562,-0.974595,0.159428,0.950499,0.219755,-0.475101,0.336727,-0.495698,-0.483023,...,0.274736,0.045784,-0.139596,0.222766,0.454966,0.738184,0.339183,-0.012991,0.058895,0.104131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17093,ZXDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.533725,0.820179,-0.039934,0.768685,0.024477,-0.469159,-0.121612,0.334984,-0.576109,0.011932
17094,ZYG11A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.842295,0.705307,-0.185267,0.308643,0.702930,1.426623,-0.431925,-0.104104,0.326705,-0.595518
17095,ZYX,0.438725,0.288916,-0.957959,-0.015910,0.924752,1.493170,-1.211469,-0.012567,1.072897,...,-0.335432,-0.220125,-0.334244,1.532781,-0.100618,0.756009,1.109339,0.340212,1.288777,-0.440243
17096,ZZEF1,-1.092240,0.605883,-0.942907,1.118054,1.816396,-0.234583,-1.788601,0.330666,-1.416258,...,-0.719897,-1.152449,-0.085764,-1.263359,-0.832392,-1.385757,-0.082991,-0.801099,-0.589180,-1.340205


In [8]:
gene_mut = pd.read_csv(r"C:\Users\justi\Coding\Project\Han Xu\DepMap_data\DepMap_data\CCLE_gene_mutation_trimmed_Wei.csv")
gene_mut.drop(columns=gene_mut.columns[0],inplace=True)
gene_mut

C:\Users\justi\AppData\Local\Temp\ipykernel_7444\3131860586.py:1: DtypeWarning: Columns (20,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_mut = pd.read_csv(r"C:\Users\justi\Coding\Project\Han Xu\DepMap_data\DepMap_data\CCLE_gene_mutation_trimmed_Wei.csv")


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,Chromosome,Start_position,End_position,Strand,Variant_Classification,Variant_Type,Reference_Allele,...,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID,Cell line
0,A1BG,1,37,19,58858743,58858743,+,Missense_Mutation,SNP,C,...,NaN,20:16,20:16,NaN,NaN,NaN,NaN,other non-conserving,ACH-000986,HT115_LARGE_INTESTINE
1,A1BG,1,37,19,58858810,58858810,+,Silent,SNP,C,...,NaN,10:21,14:23,34:87,NaN,NaN,22:19,silent,ACH-000988,MFE319_ENDOMETRIUM
2,A1BG,1,37,19,58858867,58858867,+,Missense_Mutation,SNP,C,...,NaN,4:9,4:9,NaN,NaN,NaN,NaN,other non-conserving,ACH-002182,OVCA433_OVARY
3,A1BG,1,37,19,58858872,58858872,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,9:16,other non-conserving,ACH-000985,LS411N_LARGE_INTESTINE
4,A1BG,1,37,19,58858914,58858914,+,Missense_Mutation,SNP,C,...,27:20,NaN,NaN,NaN,NaN,NaN,NaN,other non-conserving,ACH-001793,LPS27_SOFT_TISSUE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296232,ZZZ3,26009,37,1,78098868,78098868,+,Missense_Mutation,SNP,G,...,NaN,49:260,48:248,NaN,NaN,NaN,NaN,other non-conserving,ACH-002280,NB13_AUTONOMIC_GANGLIA
1296233,ZZZ3,26009,37,1,78098935,78098935,+,Silent,SNP,A,...,NaN,NaN,NaN,6:15,NaN,NaN,NaN,silent,ACH-000992,BICR18_UPPER_AERODIGESTIVE_TRACT
1296234,ZZZ3,26009,37,1,78098970,78098970,+,Frame_Shift_Del,DEL,A,...,NaN,31:8,40:87,11:64,NaN,NaN,6:16,damaging,ACH-000985,LS411N_LARGE_INTESTINE
1296235,ZZZ3,26009,37,1,78098986,78098986,+,Silent,SNP,G,...,NaN,84:28,90:33,57:46,NaN,NaN,29:12,silent,ACH-000665,SKMES1_LUNG


In [78]:
gene_mut_t = mut_trim(gene_mut, shRNA)
gene_mut_t.to_csv(r"C:\Users\justi\Coding\Project\Han Xu\Trimmed data\CRISPR+shRNA_shRNA_t.csv", index=False)

,Unnamed: 0,22RV1_PROSTATE,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,786O_KIDNEY,A1207_CENTRAL_NERVOUS_SYSTEM,A172_CENTRAL_NERVOUS_SYSTEM,A204_SOFT_TISSUE,A2058_SKIN,A549_LUNG,A673_BONE,...,UACC812_BREAST,WM115_SKIN,WM1799_SKIN,WM2664_SKIN,WM793_SKIN,WM88_SKIN,WM983B_SKIN,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,ZR751_BREAST
0,A1BG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A1CF,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,A2M,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A2ML1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A4GALT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17093,ZXDC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17094,ZYG11A,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17095,ZYX,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17096,ZZEF1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
gene_eff_t, shRNA_t = trim(gene_eff, shRNA)

AttributeError: Can only use .str accessor with string values!

In [36]:
gene_eff_t.to_csv(r"C:\Users\justi\Coding\Project\Han Xu\Trimmed data\CRISPR+shRNA_CRISPR_t.csv", index=False)
shRNA_t.to_csv(r"C:\Users\justi\Coding\Project\Han Xu\Trimmed data\CRISPR+shRNA_shRNA_t.csv", index=False)

In [30]:
gene_eff_t

,Unnamed: 0,143B_BONE,2313287_STOMACH,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,769P_KIDNEY,786O_KIDNEY,A2058_SKIN,A375_SKIN,A3KAW_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,A427_LUNG,...,WM115_SKIN,WM1799_SKIN,WM2664_SKIN,WM793_SKIN,WM88_SKIN,WM983B_SKIN,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR751_BREAST
0,A1BG,0.221363,0.059425,0.077280,-0.008899,-0.002074,0.123963,0.194133,0.281110,0.105032,...,0.166349,0.164279,0.152583,0.106293,0.404874,0.164526,0.050527,0.120837,0.201739,0.137681
1,A1CF,0.111622,0.060949,0.003774,0.223585,0.061060,0.012316,0.182061,-0.154732,0.014355,...,0.383316,0.245308,0.038037,0.032311,0.209277,0.189753,-0.004612,0.069835,0.017239,0.116339
2,A2M,-0.058403,0.058058,0.037077,-0.225029,-0.124857,-0.047788,0.018968,0.028900,-0.149137,...,-0.283927,-0.321828,-0.145072,0.004775,-0.081111,-0.093612,-0.106906,-0.146562,0.051145,0.028765
3,A2ML1,0.081484,0.215465,0.361702,0.100293,0.240875,0.130156,0.164922,0.317412,0.131668,...,0.369492,0.215881,0.204610,0.213186,0.429039,0.126596,0.022748,0.142939,0.235127,0.252360
5,A4GALT,-0.120979,0.112064,0.134559,0.015857,0.132054,-0.035362,-0.011836,-0.044328,-0.132368,...,-0.231987,-0.014898,-0.210552,-0.058201,-0.004924,-0.157073,0.074254,-0.106589,-0.148768,-0.101166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18113,ZXDC,0.100038,0.222195,0.128398,0.147377,0.121127,0.079314,0.196391,0.314431,-0.036813,...,0.169058,0.006841,-0.004628,0.040247,0.178972,0.126129,-0.193007,0.037980,0.094346,0.122096
18114,ZYG11A,-0.041764,0.065707,-0.077705,0.130222,0.086999,0.147437,0.097110,-0.112686,0.007216,...,0.083581,0.119420,0.220424,0.156121,0.196915,0.043602,0.091219,-0.107943,0.019625,0.131042
18116,ZYX,0.233361,0.276253,-0.072698,0.018974,-0.023919,-0.030598,0.082309,0.288043,0.112159,...,0.134751,0.102875,0.211251,0.032817,0.170367,0.236416,0.284462,0.128135,0.025008,0.085543
18117,ZZEF1,-0.164403,-0.260490,-0.137998,-0.305927,-0.110405,-0.270778,-0.219113,-0.383834,-0.104717,...,-0.140053,-0.185159,-0.225514,0.031118,0.162049,0.181676,-0.050515,-0.096626,-0.417338,-0.283654


In [31]:
shRNA_t

,Unnamed: 0,143B_BONE,2313287_STOMACH,697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,769P_KIDNEY,786O_KIDNEY,A2058_SKIN,A375_SKIN,A3KAW_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,A427_LUNG,...,WM115_SKIN,WM1799_SKIN,WM2664_SKIN,WM793_SKIN,WM88_SKIN,WM983B_SKIN,YD38_UPPER_AERODIGESTIVE_TRACT,YD8_UPPER_AERODIGESTIVE_TRACT,YKG1_CENTRAL_NERVOUS_SYSTEM,ZR751_BREAST
0,A1BG,0.668865,0.563500,0.407828,-0.085012,-0.378410,-0.595518,-0.518673,-0.364863,0.590831,...,0.047725,0.529830,-0.684247,0.248592,0.155388,0.589643,1.137463,-0.171007,-0.367834,-0.001838
1,A1CF,-0.509562,0.232273,1.173509,-1.144923,-0.170096,-2.284927,-1.414277,-2.272648,4.187510,...,-1.222956,-0.422814,-2.264726,-0.239732,-2.279778,0.216230,-2.425942,0.511213,1.406026,-0.160431
2,A2M,-0.261082,0.334944,0.077275,0.051528,0.202445,-0.287543,1.719349,0.707288,-0.604233,...,0.161408,0.168103,0.419712,0.609845,1.260653,-0.476685,0.442686,0.180620,0.416939,0.500914
3,A2ML1,-0.366804,0.281390,0.463284,-0.733364,-0.687812,0.324843,-0.730592,0.732243,0.178005,...,-0.350801,0.169529,0.789282,0.423673,0.353363,-0.441035,-0.233830,0.056519,-0.662857,0.940200
4,A4GALT,-0.345850,0.270695,-0.974595,-0.375915,0.159428,0.336727,0.285430,0.094624,0.715210,...,0.045784,-0.139596,0.222766,0.454966,0.738184,0.339183,-0.012991,0.058895,-0.299465,0.104131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17093,ZXDC,0.628066,-0.126722,NaN,-0.164352,NaN,NaN,-0.313012,0.869297,0.506460,...,0.820179,-0.039934,0.768685,0.024477,-0.469159,-0.121612,0.334984,-0.576109,NaN,0.011932
17094,ZYG11A,0.043566,-0.497679,NaN,-0.025068,NaN,NaN,0.126986,1.049923,-1.537071,...,0.705307,-0.185267,0.308643,0.702930,1.426623,-0.431925,-0.104104,0.326705,NaN,-0.595518
17095,ZYX,-0.023310,0.001883,0.288916,-0.172473,-0.957959,-1.211469,-1.253457,0.423673,-0.657707,...,-0.220125,-0.334244,1.532781,-0.100618,0.756009,1.109339,0.340212,1.288777,-0.338562,-0.440243
17096,ZZEF1,-1.232859,1.117658,0.605883,-1.962494,-0.942907,-1.788601,-0.272649,0.779776,0.441894,...,-1.152449,-0.085764,-1.263359,-0.832392,-1.385757,-0.082991,-0.801099,-0.589180,2.204979,-1.340205


In [41]:
var_class = gene_mut.iloc[:,7]
var_class.unique()

array(['Nonsense_Mutation', 'In_Frame_Del', 'Silent', 'Frame_Shift_Ins',
       'Missense_Mutation', 'Splice_Site', 'Frame_Shift_Del',
       'De_novo_Start_OutOfFrame', 'Nonstop_Mutation', 'In_Frame_Ins',
       'Start_Codon_SNP', 'Start_Codon_Del', 'Stop_Codon_Ins',
       'Start_Codon_Ins', 'Stop_Codon_Del', 'Intron', 'IGR', "5'Flank",
       "3'UTR", "5'UTR"], dtype=object)